	信息增益

In [1]:
import pandas as pd
from scipy.io import arff
from math import log
import numpy as np

def calcShannonEnt(dataSet):#计算给定数据集的经验熵(香农熵)
    numEntires = len(dataSet)                     
    labelCounts = {}                                 #保存每个标签(Label)出现次数的字典
    for featVec in dataSet:                        
        currentLabel = featVec[-1]                   
        if currentLabel not in labelCounts.keys():  
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1              
    shannonEnt = 0.0                                 #经验熵(香农熵)
    for key in labelCounts:                       
        prob = float(labelCounts[key]) / numEntires  #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2)            #利用公式计算
    return shannonEnt                                #返回经验熵(香农熵)

def splitDataSet(dataSet, axis, value):#函数说明:按照给定特征划分数据集
    retDataSet = []                                 
    for featVec in dataSet:                          
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]             #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])     #将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet                                  

def chooseBestFeatureToSplit(dataSet):#函数说明:选择最优特征
    numFeatures = len(dataSet[0]) - 1                     #特征数量
    baseEntropy = calcShannonEnt(dataSet)                 #计算数据集的香农熵
    lis = []
    for i in range(numFeatures):                        
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        featList = pd.cut(featList, 20).codes
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        newEntropy = 0.0                                   #经验条件熵
        for value in uniqueVals:                          
            subDataSet = splitDataSet(dataSet, i, value)           #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))        
            newEntropy += prob * calcShannonEnt(subDataSet)        #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy                        #信息增益
        lis.append(infoGain)
    return lis

In [2]:
data = arff.loadarff('/home/aistudio/data/data61550/CM1.arff')
df = pd.DataFrame(data[0])
data = df.values.tolist()
bestfeature=chooseBestFeatureToSplit(data)
m = sorted(bestfeature, reverse = True)
lis = []
for i in m[0:6]:
    for j in range(len(bestfeature)):
        if i==bestfeature[j]:
            lis.append(j)
            bestfeature[j] = -1
            break
print("排行前6个特征：")
for i in lis:
    print(df.columns[i])

排行前6个特征：
CYCLOMATIC_DENSITY
HALSTEAD_CONTENT
HALSTEAD_DIFFICULTY
HALSTEAD_EFFORT
HALSTEAD_ERROR_EST
HALSTEAD_LENGTH


	卡方

In [3]:
from scipy.stats import chi2_contingency

data = df.values.T.tolist()
ans = []
for j in range(len(data)-1):
    l = []
    l.append(data[j])
    l.append(data[-1])
    for i in range(len(data[-1])):
        if data[-1][i]=='N':
            data[-1][i] = 0
        else:
            data[-1][i] = 1
    chi2,pval,dof,expected = chi2_contingency(l)
    ans.append(pval)
m = sorted(ans, reverse = True)
lis = []
for i in m[0:6]:
    for j in range(len(ans)):
        if i==ans[j]:
            lis.append(j)
            ans[j] = -1
            break
print("排行前6个特征：")
for i in lis:
    print(df.columns[i])

排行前6个特征：
CYCLOMATIC_DENSITY
DECISION_DENSITY
DESIGN_DENSITY
ESSENTIAL_DENSITY
HALSTEAD_DIFFICULTY
HALSTEAD_ERROR_EST


	relief

In [4]:
import numpy as np
from random import randrange

def relief(features, labels,times):   
    (n_samples, n_features) = np.shape(features)
    delta = []
    delta_features = []
    delta_index = []
    sample_distance = sap_distance(features)  # 计算每两个样本之间的距离
    new_features = normalize(features)        # 对特征值归一化
 
    '''# 下面开始计算相关统计量，并对各个特征的相关统计量进行比较，最后返回各个特征值相关统计量从高到低的排名
    # 这是将随机选取的样本代入计算出来的delta
    for i in range(0, times):
        randnum = randrange(0, n_samples, 1)        # 生成一个随机数
        one_sample = features[randnum]        # 随机选择一个样本
        (nearhit, nearmiss, nearhit_index, nearmiss_index) = search_near(sample_distance, labels, randnum, features)  # 找出猜中近邻和猜错近邻,nearhit为猜中近邻样本的行向量
        delta.append(relevant_feature(nearhit_index, nearmiss_index, new_features, randnum))  # 计算相关统计量矩阵
    delta = np.asarray(delta)
    for j in range(0, n_features):
        delta_features.append(np.sum(delta[:, j]))
    midd = list(set(delta_features))
    midd.sort(reverse=True)
    for p in midd:
        for q in range(0, len(delta_features)):
            if delta_features[q] == p:
                delta_index.append(q)
    return delta_index'''
    # 这是将所有样本都带入计算的delta
    for i in range(0, n_samples):
        (nearhit, nearmiss, nearhit_index, nearmiss_index) = search_near(sample_distance, labels, i,
                                                                         features)  # 找出猜中近邻和猜错近邻,nearhit为猜中近邻样本的行向量
        delta.append(relevant_feature(nearhit_index, nearmiss_index, new_features, i))  # 计算相关统计量矩阵
    delta = np.asarray(delta)
    for j in range(0, n_features):
        delta_features.append(np.sum(delta[:, j]))
    midd = list(set(delta_features))
    midd.sort(reverse=True)
    for p in midd:
        for q in range(0, len(delta_features)):
            if delta_features[q] == p:
                delta_index.append(q)
    return delta_index


def normalize(features):
    (n_samples, n_features) = np.shape(features)
    fe_max = []
    fe_min = []
    n_deno = []
    new_features = np.zeros((n_samples, n_features))
    for i in range(0, n_features):
        max_index = np.argmax(features[:, i])
        min_index = np.argmin(features[:, i])
        fe_max.append(features[max_index, i])  # 计算每一个特征的最大值
        fe_min.append(features[min_index, i])  # 计算每一个特征的最小值
    n_deno = np.asarray(fe_max) - np.asarray(fe_min)  # 求出归一化的分母
    for j in range(0, n_features):
        for k in range(0, n_samples):
            new_features[k, j] = (features[k, j]-fe_min[j]) / n_deno[j]  # 归一化
    return new_features

def sap_distance(features):
    (n_samples, n_features) = np.shape(features)
    distance = np.zeros((n_samples, n_samples))
    for i in range(0, n_samples):
        for j in range(0, n_samples):
            diff_distance = features[i]-features[j]
            if i == j:
                distance[i, j] = 9999
            else:
                distance[i, j] = euclid_distance(diff_distance)  # 使用欧几里德距离定义样本之间的距离
    return distance

def euclid_distance(diff_distance):
    counter = np.power(diff_distance, 2)
    counter = np.sum(counter)
    counter = np.sqrt(counter)
    return counter

def search_near(sample_distance, labels, randnum, feartures):
    (n_samples, n_features) = np.shape(feartures)
    nearhit_list = []
    nearmiss_list = []
    hit_index = []
    miss_index = []
    for i in range(0, n_samples):
        if labels[i] == labels[randnum]:
            nearhit_list.append(sample_distance[i, randnum])  # 将距离放在一个列表里面
            hit_index.append(i)                                 # 将样本标号放在另一个列表里面
        else:
            nearmiss_list.append(sample_distance[i, randnum])
            miss_index.append(i)
    nearhit_dis_index = nearhit_list.index(min(nearhit_list))   # 算出猜中近邻
    nearhit_index = hit_index[nearhit_dis_index]                # 将猜中近邻的样本标号赋给nearhit_index

    nearmiss_dis_index = nearmiss_list.index(min(nearmiss_list))
    nearmiss_index = miss_index[nearmiss_dis_index]


    nearhit = feartures[nearhit_index]
    nearmiss = feartures[nearmiss_index]

    return nearhit, nearmiss, nearhit_index, nearmiss_index

def relevant_feature(nearhit_index, nearmiss_index, new_features, randnum):
    diff_hit = abs(new_features[nearhit_index]-new_features[randnum])
    diff_miss = abs(new_features[nearmiss_index]-new_features[randnum])
    delta = -np.power(diff_hit, 2)+np.power(diff_miss, 2)
    return delta

In [5]:
data = df.values.T
labels = data[-1]
features = data[:-1].T

features_importance = relief(features, labels,3)
print("排行前6个特征：")
for i in features_importance[:6]:
    print(df.columns[i])

排行前6个特征：
PERCENT_COMMENTS
DESIGN_DENSITY
CALL_PAIRS
MAINTENANCE_SEVERITY
HALSTEAD_CONTENT
LOC_CODE_AND_COMMENT
